### RULES
Requires format of skipped line between different phylums (ex. empty row above Diatom, Dinoflagellate, etc.)

Assumed all {Ochromonas, } are mixotrophs.

1. assume everything after "Unknown flagellates" is irrelevant (to be deleted)
2. diatoms are NOT mixotrophs
3. remove all "[name]-like" (without genus specified)
4. remove all "[genus name] spp." AND "[genus name] sp."
5. check "cysts of"

Status Key--  
Confirmed := explicitly in the Mixotroph Database  
Unsure (sp. in mdb) := genus in Mixotroph Database lists "[genus name] sp." (ex. Ochromonas sp. for Ochromonas danica)  
Unsure (inexact name):= LIS name is in a longer Mixotroph Database name or vice versa (ex. Chattonella marina in Chattonella marina var. ovata)   

### QUESTIONS TO ASK

1. Should I be considering "cysts of Linggulodinium polyedrum" mixotrophs?
2. How should I handle these "unsure" cases? - see status key above

In [152]:
import pandas as pd
import numpy as np

from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option("future.no_silent_downcasting", True)

In [153]:
mdb = pd.read_csv("MDB - 3Dec2022.csv")
mdb.columns = mdb.iloc[1]
mdb = mdb.drop([0, 1]).reset_index(drop=True)

# edit mdb so that species ending in "sp" now end in "sp."
mdb['Species Name'] = mdb['Species Name'].str.replace(r'sp$', 'sp.', regex=True)

mdb.head()

1,Species Name,Taxonomic Group,AphiaID,Additional notes,Gene markers,PR2 Accession Number,GenBank Accession Number,Reference to sequence,MFT,Evidence of mixoplankton activity,Reference to mixoplankton activity,can form colonies,has benthic stages,has parasitic stages,HABs,L (μm),W (μm) or diameter (μm),Depth (μm) or other information,size class,colony size,Reference to mixoplankton size,ability to use NO3,Reference to NO3 usage,Mode of feeding,Presumed prey size,tested min size (μm),tested max size (μm),ingested min size (μm),ingested max size (μm),# prey spp tested,Foram ontogenetic stage (20 μm),presumed prey size class for col AG,Foram ontogenetic stage (20-200 μm),Presumed prey size (µm) for col AI,Foram ontogenetic stage (200 μm - adult),Presumed prey size (µm) for col AK,References for prey sizes,example prey types,prey taxonomic group,References for prey types,Plastid source,plastid taxonomic group,indicative size in environment,References for plastids,Symbiont source,Symbiont taxonomic group,References for taxonomic information,symbiont size (environment),symbiont size (in hospite),indicative size,symbionts per host,References and additional notes for symbionts,NaN,ALSK,ANTA,APLR,ARAB,ARCH,ARCT,AUSE,AUSW,BENG,BERS,BPLR,BRAZ,CAMR,CARB,CCAL,CHIL,CHIN,CNRY,EAFR,ETRA,FKLD,GFST,GUIA,GUIN,INDE,INDW,ISSG,KURO,MEDI,MONS,NADR,NASE,NASW,NATR,NECS,NEWZ,NPPF,NPSW,NPTG,NWCS,PEQD,PNEC,PSAE,PSAW,REDS,SANT,SARC,SATL,SPSG,SSTC,SUND,TASM,WARM,WTRA
0,Acanthochiasma sp.,Radiolaria,368427,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,HM103395.1.1099_U;HM103418.1.1104_U;JN811207.1...,HM103395;HM103418;JN811207;GU825020;HM103399;H...,"Quaiser,A.. Comparative metagenomics of bathyp...",eSNCM,endosymbionts,"Decelle J, Siano R, Probert I et al (2012b) Mu...",no,not recorded,not recorded,not recorded,100,150,not recorded,micro,not applicable,"Decelle J, Siano R, Probert I et al (2012b) Mu...",not recorded,this work,prey capture using axopodia; engulfment after ...,pico-micro,not recorded,not recorded,not recorded,not recorded,not recorded,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,"Swanberg, N.R. and Caron, D.A., 1991. Patterns...","tintinnids, mollusc larvae, copepods, ostracod...","Diatomeae, Ciliophora, Copepoda, Mollusca, Ost...","Swanberg, N.R. and Harbison, G.R., 1980. The e...",not applicable,not applicable,not applicable,not applicable,"Chrysochromulina, Azadinium sp., Pelagodinium ...","Haptophyta, Dinoflagellata","Decelle J, Siano R, Probert I et al (2012b) Mu...",not recorded,4-15 um,nano,5 (<100 μm); 10-29 (100-200 μm); 29-45 (200-30...,"Decelle J, Siano R, Probert I et al (2012b) Mu...",-,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded
1,Acanthometra fusca,Radiolaria,not registered,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,KC172856.1.1696_U;EU446351.1.1552_U;JN811165.1...,KC172856;EU446351;JN811165,"Decelle,J.. Diversity, ecology and biogeochemi...",eSNCM,endosymbionts,"Michaels, A.F., 1988. Vertical distribution an...",no,not recorded,not recorded,not recorded,100-300,100-300,not recorded,micro-meso,not applicable,"Michaels, A.F., 1988. Vertical distribution an...",not recorded,this work,prey capture using axopodia; engulfment after ...,pico-micro,not recorded,not recorded,not recorded,not recorded,not recorded

In [154]:
csv_name = "LIS_2019-Phytoplankton_Final Report Data.xlsx - 2019 LIS phytoplankton count"

In [155]:
# import and clean LIS data
lis = pd.read_csv(f"inputs/{csv_name}.csv")
original_headers = lis.columns  # save original column headers

phylum_ind = lis[lis.iloc[:, 0] == "Phylum"].index[0]
lis.columns = lis.iloc[phylum_ind]  # reset column headers
lis = lis.iloc[phylum_ind+2:].reset_index(drop=True)  

In [156]:
# remove rows after unknown flagellates
unknown_flagellates_ind = lis[lis["Phylum"] == "Unknown flagellates"].index[0] 
lis = lis.iloc[:unknown_flagellates_ind]
lis = lis.iloc[:lis.last_valid_index()+1]  # remove trailing nan rows

In [157]:
# remove rows that contain "TOTAL"
lis = lis[~lis["Phylum"].str.contains("TOTAL", na=False)].reset_index(drop=True)  

In [158]:
# construct correct phylum column
actual_phylum_ind = lis[lis["Species"].isna() & lis["Phylum"].isna()].index + 1
lis = lis.rename(columns={"Phylum": "Genus"}) # rename phylum column to genus
lis.insert(0, 'Phylum', lis["Genus"].iloc[actual_phylum_ind])  # reconstruct phylum column
lis['Phylum'] = lis['Phylum'].ffill()  # forwardfill phylum

lis['Genus'] = lis['Species'].str.split().str[0]  # fill genus using first word of species name

lis = lis.dropna(subset=['Species']).reset_index(drop=True) # delete rows with na in Species column

SPECIES_COL = lis.columns.get_loc("Species")
lis.iloc[:, SPECIES_COL+1:] = lis.iloc[:, SPECIES_COL+1:].replace(",", "", regex=True).astype(float)  # ensure numerical values are floats

In [159]:
# add Status column
lis.insert(0, 'Status', None)

In [160]:
# store blocks of known mixotroph genuses 
ochromonas_ind = lis[lis["Species"].str.contains("Ochromonas")].index
ochromonas_block = lis.iloc[ochromonas_ind] 

In [161]:
# remove based on hard coded rules (NOT RESETTING INDEX IN ORDER TO ADD BLOCKS BACK CORRECTLY)
lis = lis[lis["Phylum"] != "Diatom"] # remove all diatoms
lis = lis[~lis["Species"].str.contains("-like")] # remove species ending with "-like"
lis = lis[~lis["Species"].str.contains("sp.|spp.")]  # remove all sp. / spp.

In [162]:
# check "cysts of"
CYSTS_LEN = len("cysts of ")
cysts_of = lis[lis["Species"].str.contains("cysts of", regex=False)]["Species"].str.slice(CYSTS_LEN)
filtered = cysts_of.isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Confirmed"
lis.loc[filtered[filtered].index, "Genus"] = cysts_of.str.split().str[0]

In [163]:
# add back stored blocks of known mixotrophs and mark as Confirmed
lis = pd.concat([lis, ochromonas_block]).sort_index().drop_duplicates()
lis.loc[ochromonas_ind, "Status"] = "Confirmed"

In [165]:
# check if (in none status) direct match and mark all Trues as "Confirmed"
filtered = lis["Species"].isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Confirmed"

# check (in remaining none status) if the genus has sp. and mark all Trues as "Unsure (sp. in mdb)"
genus_to_check = lis[lis['Status'].isnull()]['Species'].str.split().str[0].drop_duplicates() + " sp."
filtered = genus_to_check.isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Unsure (sp. mdb)"

In [166]:
# check (in remaining none status) if the name is contained in the mdb and vice versa and mark all Trues as "Unsure (inexact name)"
filtered = lis[lis['Status'].isnull()]["Species"].apply(lambda x: mdb["Species Name"].str.contains(x, regex=False).any())
lis.loc[filtered[filtered].index, "Status"] = "Unsure (inexact name)"

pattern = '|'.join(mdb['Species Name'])
filtered = lis[lis['Status'].isnull()]["Species"].str.contains(pattern, regex=True)
lis.loc[filtered[filtered].index, "Status"] = "Unsure (inexact name)"

In [167]:
# drop all rows with Status = "None"
lis = lis.dropna(subset=['Status']).reset_index(drop=True)

In [178]:
lis = pd.merge(lis, mdb[['Species Name','MFT', 'Evidence of mixoplankton activity', 'size class']], left_on='Species', right_on='Species Name', how='left').drop(columns=['Species Name']).reset_index(drop=True) 
lis = lis[lis.columns[:4].to_list() + lis.columns[-3:].to_list() + lis.columns[4:-3].to_list()]
lis

1,Status,Phylum,Genus,Species,MFT_y,Evidence of mixoplankton activity_y,size class_y,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/6/19,2/6/19,2/6/19,...,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/5/19,12/5/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/5/19,12/5/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,MFT_x,Evidence of mixoplankton activity_x,size class_x
0,Confirmed,Dinoflagellate,Akashiwo,Akashiwo sanguinea,CM,"ingestion of ciliates, Isochrysis, Cryptophyte...",micro,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [151]:
filtered_mdb = mdb[['Species Name','MFT', 'Evidence of mixoplankton activity', 'size class']].loc[mdb['Species Name'].isin(lis['Species'])]
lis1 = pd.merge(lis, filtered_mdb, left_on='Species', right_on='Species Name', how='left').drop(columns='Species Name')
filtered_mdb

1,Species Name,MFT,Evidence of mixoplankton activity,size class
12,Akashiwo sanguinea,CM,"ingestion of ciliates, Isochrysis, Cryptophyte...",micro
111,Dinophysis acuminata,pSNCM,photosynthetic Dinophysis spp. obtain plastids...,micro
116,Dinophysis miles,pSNCM*,This species retains chloroplasts from cryptop...,micro
117,Dinophysis norvegica,pSNCM,photosynthetic Dinophysis spp. obtain plastids...,micro
147,Gambierdiscus toxicus,CM,"presence of feeding vacuoles, unknown prey",micro
167,Gonyaulax polygramma,CM,"ingestion of cryptophyte species, Amphidinium ...",micro
186,Heterocapsa circularisquama,CM,bacteria in food vacuoles,nano
189,Heterosigma akashiwo,CM,uptake of eubacteria,micro
235,Lingulodinium polyedra,CM,"ingestion of Skeletonema costatum, Synechococcus",micro
253,Noctiluca scintillans,eSNCM,endosymbionts,meso


In [109]:
# totals = lis.groupby('Phylum', as_index=False, sort=False).sum()
# totals

# # empty text-containing columns
# totals = totals.drop("Status", axis=1)
# totals.insert(0, 'Status', "") 
# totals["Genus"] = ""
# totals["Species"] = ""

# # rename to TOTAL "   "
# totals["Phylum"] = totals["Phylum"].str.upper().apply(lambda x: "TOTAL " + x + "S")

# # add in line skips
# totals = totals.set_index(lis.groupby(['Phylum']).tail(1).index + 0.1)
# empty_df = pd.DataFrame("", index=lis.groupby(['Phylum']).tail(1).index+0.2, columns=totals.columns)
# totals = pd.concat([totals, empty_df]).sort_index()

In [171]:
# lis = pd.concat([lis, totals]).sort_index().reset_index(drop=True)

In [172]:
# replace Nans with zero
lis = lis.fillna(0)

In [173]:
# add back multiheader
needed_cols = pd.Series(np.full(len(lis.columns) - len(original_headers), None)) 
original_headers = pd.concat([needed_cols, original_headers.to_series()], ignore_index=True)
lis.columns = pd.MultiIndex.from_arrays([original_headers, lis.columns])

In [174]:
lis


NaN                                 \
1                  Status                         Phylum   
0               Confirmed                 Dinoflagellate   
1               Confirmed                 Dinoflagellate   
2               Confirmed                 Dinoflagellate   
3               Confirmed                 Dinoflagellate   
4               Confirmed                 Dinoflagellate   
5               Confirmed                 Dinoflagellate   
6               Confirmed                 Dinoflagellate   
7               Confirmed                 Dinoflagellate   
8               Confirmed                 Dinoflagellate   
9               Confirmed                 Dinoflagellate   
10              Confirmed                 Dinoflagellate   
11  Unsure (inexact name)                   Raphidophyte   
12              Confirmed                   Raphidophyte   
13              Confirmed  Ochrophyte (diatoms excluded)   
14              Confirmed  Ochrophyte (diatoms excluded)   
15              Confirmed                     Haptophyte   

   (Note: S: surface water sample, B: bottom water sample  \
1                                                   Genus   
0                                            Akashiwo       
1                                          Dinophysis       
2                                          Dinophysis       
3                                          Dinophysis       
4                                       Gambierdiscus       
5                                           Gonyaulax       
6                                         Heterocapsa       
7                                           Noctiluca       
8                                        Prorocentrum       
9                                        Prorocentrum       
10                                      Lingulodinium       
11                                        Chattonella       
12                                        Heterosigma       
13                                         Ochromonas       
14                                         Ochromonas       
15                                        Phaeocystis       

                         Unnamed: 1 A4S Note: Skeletonema bloom      B3S  \
1                           Species                      1/3/19   1/3/19   
0                Akashiwo sanguinea                           0        0   
1              Dinophysis acuminata                           0        0   
2                  Dinophysis miles                           0        0   
3              Dinophysis norvegica                           0        0   
4             Gambierdiscus toxicus                         352        0   
5              Gonyaulax polygramma                           0        0   
6       Heterocapsa circularisquama                       17600  17600.0   
7             Noctiluca scintillans                           0        0   
8                 Prorocentrum lima                           0        0   
9               Prorocentrum micans                           0        0   
10  cysts of Lingulodinium polyedra                           0        0   
11               Chattonella marina                           0        0   
12             Heterosigma akashiwo                           0        0   
13                  Ochromonas spp.                           0        0   
14                Ochromonas danica                           0        0   
15              Phaeocystis globosa                           0        0   

   C1S Note: low cell abundance     D3S E1S Note: low cell abundance  \
1                        1/3/19  1/7/19                       1/7/19   
0                             0       0                            0   
1                             0       0                            0   
2                             0       0                            0   
3                             0       0                            0   
4                             0       0  

In [41]:
# save dataframe to excel
lis.to_excel(f"outputs/{csv_name}-{str(datetime.now())}.xlsx")